# Week 1 Optimisation

## Summary

The objective for this week is to come up with an initial team to start of the 2020 season.

Stats we will be taking into account in our objective functions we are trying to maximise within the constraints:
- m1. Average Fantasy Points in 2019 season (weight: 45%)
- m2. Total Fantasy Points in this race last year (weight: 15%)
- m3. F1 experts rating of the teams after preseason testing (20%)
- m4. Team standings after FP2 (20%)

How important each of the above metric was determined arbitrarily based on how much importance I place on them myself.This is primarily done by the fact you can only make one change per week, and so I am prioritising performance over the whole season (m1 & m3) over potential performance that race (m2 & m4).

Probems with this apprach are it is heavily reliant on last year being representive of this year and all forward looking data points are not very representive of the race.

In [1]:
import requests
import pandas as pd
from pulp import *
import re

## Get Data

Get Cost Data

In [2]:
r = requests.get('https://fantasy-api.formula1.com/partner_games/f1/players')

In [3]:
cost = pd.DataFrame(r.json()['players'] )
cost = cost.set_index('id').reset_index(drop=True)

In [4]:
cost.head(2)

,first_name,last_name,team_name,position,position_id,position_abbreviation,price,current_price_change_info,status,injured,...,season_score,driver_data,constructor_data,born_at,season_prices,price_change,num_fixtures_in_gameweek,has_fixture,display_name,external_id
0,Scuderia Ferrari Mission Winnow,None,Scuderia Ferrari Mission Winnow,Constructor,2,CR,27.4,None,None,False,...,0,None,"{'best_finish': 1, 'best_finish_count': 239, '...",None,None,27.4,2,True,Ferrari,526:59:FER
1,ROKiT Williams Racing,None,ROKiT Williams Racing,Constructor,2,CR,6.5,None,None,False,...,0,None,"{'best_finish': 1, 'best_finish_count': 114, '...",None,None,6.5,2,True,Williams,481:172:WIL


Get 2019 fantasy points data and merge with cost data to get final data set for optimisation

In [5]:
driver = pd.read_pickle('../2019_Driver_Points.pickle')

In [6]:
constructor = pd.read_pickle('../2019_Constructor_Points.pickle')

In [11]:
testing = pd.read_pickle('../2020_Testing.pickle')

In [7]:
cost.loc[cost.position=='Driver', 'display_name']=cost[cost.position=='Driver']['display_name'].str.replace(' ', '')

In [8]:
driver_data = cost[['display_name','position','price', 'team_abbreviation']][cost.position=='Driver'].merge(
    driver[['Name', 'AvgPoints', 'Mel']],
    left_on = 'display_name',
    right_on = 'Name',
    how='left'
).drop(columns=['Name'])

In [12]:
driver_data['AvgPoints'] = driver_data.AvgPoints.fillna(0)
driver_data['Mel'] = driver_data.Mel.fillna(0)

In [13]:
driver_data.head(2)

,display_name,position,price,team_abbreviation,AvgPoints,Mel
0,L.Hamilton,Driver,31.3,MER,40.05,32.0
1,V.Bottas,Driver,28.4,MER,31.48,48.0


In [14]:
constructor_data = cost[['display_name', 'position','team_abbreviation', 'price']][cost.position=='Constructor'].merge(
    constructor[['Constructor','team_abbreviation', 'AverageCon', 'Mel']],
    on='team_abbreviation',
    how='left'
).drop(columns=['Constructor']).rename(
    columns={
        'AverageCon': 'AvgPoints',
    }
)

In [15]:
constructor_data['AvgPoints'] = constructor_data.AvgPoints.fillna(0)
constructor_data['Mel'] = constructor_data.Mel.fillna(0)

In [16]:
constructor_data.head(2)

,display_name,position,team_abbreviation,price,AvgPoints,Mel
0,Ferrari,Constructor,FER,27.4,45.8,42
1,Williams,Constructor,WIL,6.5,11.1,16


In [20]:
data = pd.concat(
    [constructor_data, driver_data]
).merge(
    testing[['team_abbreviation', 'testing']],
    on='team_abbreviation'
).drop(columns=['team_abbreviation']).reset_index(drop=True)

In [24]:
data.head(2)

,display_name,position,price,AvgPoints,Mel,testing
0,Ferrari,Constructor,27.4,45.80,42.0,8
1,S.Vettel,Driver,21.8,24.05,27.0,8


In [25]:
data.to_pickle('week1_data.pickle')

Standerdise features by removing the mean and scaling to unit variance

In [44]:
def z_score(series):
    mean = series.mean()
    std = series.std()
    
    return (series - mean) / std

In [45]:
data.AvgPoints = z_score(data.AvgPoints)
data.Mel = z_score(data.Mel)
data.testing = z_score(data.testing)

In [47]:
data.head(2)

,display_name,position,price,AvgPoints,Mel,testing
0,Ferrari,Constructor,27.4,1.890751,1.399528,0.855759
1,S.Vettel,Driver,21.8,0.445053,0.579492,0.855759


## Optimisation - Raw

Create Decison Variables

In [26]:
constructor_var = LpVariable.dict(
    'Constructor', 
    data['display_name'][data.position=='Constructor'], 
    0, 
    cat='Binary'
)

In [27]:
driver_var = LpVariable.dict(
    'Driver', 
    data['display_name'][data.position=='Driver'], 
    0, 
    cat='Binary'
)

Create the problem variable

In [59]:
prob = LpProblem('Fantasty_F1_Week_1', LpMaximize)

In [60]:
driver_rewards = []
constructor_rewards = []

In [61]:
driver_coef = list(
    data[data.position=='Driver'].eval(
        '0.45*AvgPoints + 0.15*Mel + 0.2*testing'
    )
)
constructor_coef = list(
    data[data.position=='Constructor'].eval(
        '0.45*AvgPoints + 0.15*Mel + 0.2*testing'
    )
)
driver_rewards += lpSum( 
    [a*b for a, b in zip(driver_coef, driver_var.values()) ]
)
constructor_rewards += lpSum( 
    [a*b for a, b in zip(constructor_coef, constructor_var.values()) ]
)

In [62]:
prob += driver_rewards + constructor_rewards, 'Total Points Scored'

Add constraints

In [63]:
prob += lpSum(driver_var) == 5, 'Max Number of Drivers'
prob += lpSum(constructor_var) == 1, 'Max Number of Constructor'

In [64]:
driver_costs = []
constructor_costs = []
driver_price = list(data[data.position=='Driver'].price)
constructor_price = list(data[data.position=='Constructor'].price)

driver_costs += lpSum(
    [a*b for a,b in zip(driver_price, driver_var.values())]
)
constructor_costs += lpSum(
    [a*b for a,b in zip(constructor_price, constructor_var.values())]
)

In [65]:
prob += driver_costs + constructor_costs <= 100, 'Total Cost of Team'

In [66]:
prob.solve()

1

In [67]:
print("Status:", LpStatus[prob.status])

Status: Optimal


### Results

In [68]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [69]:
summary (prob)

Variables:

Constructor_Mercedes = 1.0
Driver_D.Kvyat = 1.0
Driver_L.Stroll = 1.0
Driver_P.Gasly = 1.0
Driver_S.Perez = 1.0
Driver_V.Bottas = 1.0
---------------------------------------

Constraints:
32.2*1.0 + 9.9*1.0 + 7.9*1.0 + 10.4*1.0 + 9.3*1.0 + 28.4*1.0 = 98.1
---------------------------------------

Score:
2.1607768904301836*1.0 + 0.34466611265548125*1.0 + 0.1706905225852742*1.0 + 0.15505418264854248*1.0 + 0.12960894713028362*1.0 + 0.9897171189016905*1.0 = 2.3504742443122923


## Optimisation - with Hamilton & GeorgeR in the team

In [96]:
prob = LpProblem('Fantasty_F1_Week_1', LpMaximize)

In [97]:
#Objective Function
prob += driver_rewards + constructor_rewards, 'Total Points Scored'

In [98]:
#hard Constraints
prob += lpSum(driver_var) == 5, 'Max Number of Drivers'
prob += lpSum(constructor_var) == 1, 'Max Number of Constructor'
prob += driver_costs + constructor_costs <= 100, 'Total Cost of Team'

In [99]:
#Soft Constraints
prob += driver_var['L.Hamilton'] == 1, 'Hamilton has to be in the team'
prob += driver_var['G.Russell'] == 1, 'Russell has to be in the team'
prob += driver_var['P.Gasly'] == 0, 'Gasly not to be in the team' #changed team mid 2019

In [100]:
prob.solve()

1

In [101]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [102]:
summary (prob)

Variables:

Constructor_Mercedes = 1.0
Driver_D.Kvyat = 1.0
Driver_G.Russell = 1.0
Driver_L.Hamilton = 1.0
Driver_L.Stroll = 1.0
Driver_S.Perez = 1.0
---------------------------------------

Constraints:
32.2*1.0 + 9.9*1.0 + 5.9*1.0 + 31.3*1.0 + 7.9*1.0 + 9.3*1.0 = 96.5
---------------------------------------

Score:
2.1607768904301836*1.0 + 0.34466611265548125*1.0 + 0.591523854842104*1.0 + 1.1148485699697241*1.0 + 0.1706905225852742*1.0 + 0.12960894713028362*1.0 = 2.0391360231867646


### END